In [5]:
dirname(dirname(pwd()))

"/Users/alexpanchot/Documents/Caller_preds/caller_predictions"

In [6]:
import CSV
using DataFrames
# using StatsBase
include("$( dirname(dirname(pwd())) )/pred_funcs.jl")
using MLJ
# import MLJBase
import TableView.showtable
using Plots
import FileIO
import Statistics
import LinearAlgebra
using StatsPlots
using Random
Random.seed!(0);
using StatsBase
import MLJBase

WebIO._IJuliaInit()

In [7]:
MissingImputator = @load MissingImputator pkg="BetaML" verbosity=0
LogisticClassifier = @load LogisticClassifier pkg="MLJLinearModels" verbosity=0;  # loads code defining a model type
EvoTreeClassifier = @load EvoTreeClassifier verbosity=0

EvoTrees.EvoTreeClassifier

In [8]:
full = ""

""

In [26]:

sample_key2, xold, yold = FileIO.load("$(dirname(dirname(pwd())))/data$(full).jld2","sample_key","x" ,"y");


xold.col1=collect(1:nrow(xold));
println(nrow(xold))
filter!(x->x.ALTREF_I ≈ 0,xold)
println(nrow(xold))
# filter!(x->x.ALT_I>0.5 || x.REF_I>0.5,xold) # indel
# filter!(x->x.ALT_I<0.5 && x.REF_I<0.5,xold) # SNV
println(nrow(xold))
filter!(x->x.training>0.5,xold)
# select!(xold,Not("training"));
println(nrow(xold))
# filter!(x->!ismissing(x.dust_score_Mutect),xold);
# println(nrow(xold))

yold = yold[xold.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
sample_key2 = sample_key2[xold.col1]
select!(xold,Not("col1"));

94626
79620
79620
19737


In [27]:
for j in 1:ncol(xold), i in 1:nrow(xold)
    if !ismissing(xold[i,j]) && (isnan(xold[i,j]) || isinf(xold[i,j]) )
        xold[i,j] = missing
    end
end

In [28]:
train2 = 1:nrow(xold)
# test2 = 0
# train2, test2 = partition(eachindex(yold), 0.5, stratify=yold,  shuffle=true, rng=12344);

select!(xold,Not("training"));

In [29]:
sum([i==true for i in yold[train2]])

77

In [30]:
# sum([i==true for i in yold[test2]])

In [31]:
sum(.~ismissing.(xold[train2,:dust_score_Vardict]))

15179

In [32]:
# FileIO.save("testdata$(full).jld2", Dict("sample_key"=>sample_key2[test2],"x" => xold[test2,:],
#         "y" => yold[test2], 
#     ) ; compress = true
# )
FileIO.save("traindata$(full).jld2", Dict("sample_key"=>sample_key2[train2],"x" => xold[train2,:],
        "y" => yold[train2], 
    ) ; compress = true
)

# Mutect

In [33]:
x = copy(xold[train2,:]);
y = copy(yold[train2]);

In [34]:
x.col1=collect(1:nrow(x));
coldrop = [] #"FPpass","FILTER_Mutect_PASS"] # ["gt_AF_Mutect","AltFwd_Mutect","RefFwd_Mutect","AltRev_Mutect","RefRev_Mutect","gt_AD_ref_Mutect","gt_AD_alt_Mutect"]
for i in names(x)
    if i == "col1" || i=="training" || occursin( "FILTER_Mutect",i)
        continue
    end
    if length(unique(x[:,i])) < 2 || occursin("Pindel",i) || occursin("Lofreq",i) || occursin("Vardict",i) # || occursin("ALT",i) || occursin("REF",i)
        push!(coldrop,i)
#         println(i)
    end 
end
unique!(coldrop)
select!(x, Not(coldrop));
println(sum([i==true for i in y]))
# println(nrow(xold))
# dropmissing!(xold)
# println(nrow(xold))
filter!(x->!ismissing(x.dust_score_Mutect),x);
println(size(y))
# println(length(sample_key2))
println()
y = y[x.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
# sample_key2 = sample_key2[x.col1]
select!(x,Not("col1"));

println(size(x))
println(size(y))
# println(length(sample_key2))
println(sum([i==true for i in y]));


77
(19737,)

(494, 59)
(494,)
67


In [35]:
transform_log!(x,10)

In [36]:
stnd = fit_standardizer(x)
transform_standardizer!(x,stnd);

In [37]:
println(sum(ismissing.(Matrix(x))))

1


In [38]:
imputemach = machine(MissingImputator(K=5),x) 
@time fit!(imputemach,verbosity=1)

┌ Info: Training Machine{MissingImputator,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


 17.929886 seconds (47.68 M allocations: 3.110 GiB, 4.55% gc time)


Machine{MissingImputator,…} trained 1 time; caches data
  args: 
    1:	Source @462 ⏎ `Table{Union{AbstractVector{Union{Missing, Continuous}}, AbstractVector{Continuous}}}`


In [39]:
tt = MLJ.transform(imputemach,x);
for i in 1:ncol(x)
    x[:,i] = tt[i]
end

In [40]:
disallowmissing!(x);

In [41]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))


(494, 59) 67
(494,)


In [42]:
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"under",0.8);
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"over",0.2);

In [43]:
y = coerce(copy(y),OrderedFactor);

In [44]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))

(494, 59) 67
(494,)


In [48]:

r = [range(LogisticClassifier(), :fit_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalize_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalty, values=[:none,:l1,:l2]),
    range(LogisticClassifier(), :lambda, lower=0.0,upper=1.0),
    range(LogisticClassifier(), :gamma, lower=0.0,upper=1.0),
]

5-element Vector{MLJBase.ParamRange}:
 NominalRange(fit_intercept = true, false)
 NominalRange(penalize_intercept = true, false)
 NominalRange(penalty = none, l1, l2)
 NumericRange(0.0 ≤ lambda ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ gamma ≤ 1.0; origin=0.5, unit=0.5)

In [49]:
self_tune = TunedModel(
#     model=LogisticClassifier(penalty=:l1),
    model=LogisticClassifier(),
    
    resampling=StratifiedCV(nfolds=5, rng=123),

    tuning=LatinHypercube(),  #     tuning=Grid(resolution=2), 
    range = r,
    n=200,
    operation=predict_mode, # use for probabalistic 
    measure = fnr,  # use for deterministic or with predict_mode for probab 
    acceleration=CPUThreads(),
)

machbest = machine(
    self_tune,
    x,y)
fit!(machbest,verbosity=1 )



┌ Info: Training Machine{ProbabilisticTunedModel{LatinHypercube,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403
┌ Info: Attempting to evaluate 200 models.
└ @ MLJTuning /Users/alexpanchot/.julia/packages/MLJTuning/efiDR/src/tuned_models.jl:685
Evaluating over 200 metamodels:   0%[>                        ]  ETA: N/A┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Eva

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

Evaluating over 200 metamodels:  30%[=======>                 ]  ETA: 0:00:14┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinea

Evaluating over 200 metamodels:  51%[============>            ]  ETA: 0:00:09┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinea

Evaluating over 200 metamodels:  64%[================>        ]  ETA: 0:00:07┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  74%[==================>      ]  ETA: 0:00:05┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 100

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

Evaluating over 200 metamodels:  92%[======================>  ]  ETA: 0:00:02┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  92%[=======================> ]  ETA: 0:00:02┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  93%[=======================> ]  ETA: 0:00:01┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModel

Machine{ProbabilisticTunedModel{LatinHypercube,…},…} trained 1 time; caches data
  args: 
    1:	Source @659 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @439 ⏎ `AbstractVector{OrderedFactor{2}}`


In [50]:
mach = machine(
    fitted_params(machbest).best_model,
    x,y)
fit!(mach,verbosity=4 )

┌ Info: Training Machine{LogisticClassifier,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


Machine{LogisticClassifier,…} trained 1 time; caches data
  args: 
    1:	Source @906 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @243 ⏎ `AbstractVector{OrderedFactor{2}}`


In [51]:
FileIO.save("mutect$(full).jld2", Dict("mach"=>mach,"stnd" => stnd,
        "imputemach" => imputemach, 
    ) ; compress = true
)

# Vardict

In [52]:
x = copy(xold[train2,:]);
y = copy(yold[train2]);

In [53]:
x.col1=collect(1:nrow(x));
coldrop = [] #"FPpass","FILTER_Mutect_PASS"] # ["gt_AF_Mutect","AltFwd_Mutect","RefFwd_Mutect","AltRev_Mutect","RefRev_Mutect","gt_AD_ref_Mutect","gt_AD_alt_Mutect"]
for i in names(x)
    if i == "col1" || i=="training" || occursin( "FILTER_Vardict",i)
        continue
    end
    if length(unique(x[:,i])) < 2 || occursin("Pindel",i) || occursin("Lofreq",i) || occursin("Mutect",i) # || occursin("ALT",i) || occursin("REF",i)
        push!(coldrop,i)
#         println(i)
    end 
end
unique!(coldrop)
select!(x, Not(coldrop));
println(sum([i==true for i in y]))
# println(nrow(xold))
# dropmissing!(xold)
# println(nrow(xold))
filter!(x->!ismissing(x.dust_score_Vardict),x);
println(size(y))
# println(length(sample_key2))
println()
y = y[x.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
# sample_key2 = sample_key2[x.col1]
select!(x,Not("col1"));

println(size(x))
println(size(y))
# println(length(sample_key2))
println(sum([i==true for i in y]));



77
(19737,)

(15179, 52)
(15179,)
77


In [54]:
transform_log!(x,10)
stnd = fit_standardizer(x)
transform_standardizer!(x,stnd);
println(sum(ismissing.(Matrix(x))))

3517


In [55]:
imputemach = machine(MissingImputator(K=5),x) 
@time fit!(imputemach,verbosity=1)

┌ Info: Training Machine{MissingImputator,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


  9.763185 seconds (61.35 M allocations: 8.217 GiB, 17.72% gc time, 1.32% compilation time)


Machine{MissingImputator,…} trained 1 time; caches data
  args: 
    1:	Source @059 ⏎ `Table{Union{AbstractVector{Union{Missing, Continuous}}, AbstractVector{Continuous}}}`


In [56]:
tt = MLJ.transform(imputemach,x);
for i in 1:ncol(x)
    x[:,i] = tt[i]
end
disallowmissing!(x);

In [57]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"under",0.8);
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"over",0.2);

y = coerce(copy(y),OrderedFactor);

println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))

(15179, 52) 77
(15179,)
(15179, 52) 77
(15179,)


In [58]:
r = [range(LogisticClassifier(), :fit_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalize_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalty, values=[:none,:l1,:l2]),
    range(LogisticClassifier(), :lambda, lower=0.0,upper=1.0),
    range(LogisticClassifier(), :gamma, lower=0.0,upper=1.0),
]

5-element Vector{MLJBase.ParamRange}:
 NominalRange(fit_intercept = true, false)
 NominalRange(penalize_intercept = true, false)
 NominalRange(penalty = none, l1, l2)
 NumericRange(0.0 ≤ lambda ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ gamma ≤ 1.0; origin=0.5, unit=0.5)

In [59]:
self_tune = TunedModel(
#     model=LogisticClassifier(penalty=:l1),
    model=LogisticClassifier(),
    
    resampling=StratifiedCV(nfolds=5, rng=123),

    tuning=LatinHypercube(),  #     tuning=Grid(resolution=2), 
    range = r,
    n=200,
    operation=predict_mode, # use for probabalistic 
    measure = fnr,  # use for deterministic or with predict_mode for probab 
    acceleration=CPUThreads(),
)

machbest = machine(
    self_tune,
    x,y)
fit!(machbest,verbosity=1 )

mach = machine(
    fitted_params(machbest).best_model,
    x,y)
fit!(mach,verbosity=1 )


┌ Info: Training Machine{ProbabilisticTunedModel{LatinHypercube,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403
┌ Info: Attempting to evaluate 200 models.
└ @ MLJTuning /Users/alexpanchot/.julia/packages/MLJTuning/efiDR/src/tuned_models.jl:685
Evaluating over 200 metamodels:   0%[>                        ]  ETA: N/A┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ W

Evaluating over 200 metamodels:  20%[=====>                   ]  ETA: 0:04:44┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinea

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

Evaluating over 200 metamodels:  86%[=====================>   ]  ETA: 0:00:51┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  87%[=====================>   ]  ETA: 0:00:50┌ Warning: Proximal GD did not converge in 100

Evaluating over 200 metamodels:  94%[=======================> ]  ETA: 0:00:22┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinea

Machine{LogisticClassifier,…} trained 1 time; caches data
  args: 
    1:	Source @193 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @333 ⏎ `AbstractVector{OrderedFactor{2}}`


In [60]:
FileIO.save("vardict$(full).jld2", Dict("mach"=>mach,"stnd" => stnd,
        "imputemach" => imputemach, 
    ) ; compress = true
)

# Lofreq

In [61]:
x = copy(xold[train2,:]);
y = copy(yold[train2]);

In [62]:
x.col1=collect(1:nrow(x));
coldrop = [] #"FPpass","FILTER_Mutect_PASS"] # ["gt_AF_Mutect","AltFwd_Mutect","RefFwd_Mutect","AltRev_Mutect","RefRev_Mutect","gt_AD_ref_Mutect","gt_AD_alt_Mutect"]
for i in names(x)
    if i == "col1" || i=="training" || occursin( "FILTER_Lofreq",i)
        continue
    end
    if length(unique(x[:,i])) < 2 || occursin("Pindel",i) || occursin("Vardict",i) || occursin("Mutect",i) # || occursin("ALT",i) || occursin("REF",i)
        push!(coldrop,i)
#         println(i)
    end 
end
unique!(coldrop)
select!(x, Not(coldrop));
println(sum([i==true for i in y]))
# println(nrow(xold))
# dropmissing!(xold)
# println(nrow(xold))
filter!(x->!ismissing(x.dust_score_Lofreq),x);
println(size(y))
# println(length(sample_key2))
println()
y = y[x.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
# sample_key2 = sample_key2[x.col1]
select!(x,Not("col1"));

println(size(x))
println(size(y))
# println(length(sample_key2))
println(sum([i==true for i in y]));




77
(19737,)

(19591, 39)
(19591,)
77


In [63]:
transform_log!(x,10)
stnd = fit_standardizer(x)
transform_standardizer!(x,stnd);
println(sum(ismissing.(Matrix(x))))

2317


In [64]:
imputemach = machine(MissingImputator(K=5),x) 
@time fit!(imputemach,verbosity=1)

┌ Info: Training Machine{MissingImputator,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


 17.371612 seconds (77.42 M allocations: 8.643 GiB, 9.14% gc time, 52.18% compilation time)


Machine{MissingImputator,…} trained 1 time; caches data
  args: 
    1:	Source @815 ⏎ `Table{Union{AbstractVector{Union{Missing, Continuous}}, AbstractVector{Continuous}}}`


In [65]:
tt = MLJ.transform(imputemach,x);
for i in 1:ncol(x)
    x[:,i] = tt[i]
end
disallowmissing!(x);

In [66]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"under",0.8);
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"over",0.2);

y = coerce(copy(y),OrderedFactor);

println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))

(19591, 39) 77
(19591,)
(19591, 39) 77
(19591,)


In [67]:
r = [range(LogisticClassifier(), :fit_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalize_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalty, values=[:none,:l1,:l2]),
    range(LogisticClassifier(), :lambda, lower=0.0,upper=1.0),
    range(LogisticClassifier(), :gamma, lower=0.0,upper=1.0),
]

5-element Vector{MLJBase.ParamRange}:
 NominalRange(fit_intercept = true, false)
 NominalRange(penalize_intercept = true, false)
 NominalRange(penalty = none, l1, l2)
 NumericRange(0.0 ≤ lambda ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ gamma ≤ 1.0; origin=0.5, unit=0.5)

In [68]:
self_tune = TunedModel(
#     model=LogisticClassifier(penalty=:l1),
    model=LogisticClassifier(),
    
    resampling=StratifiedCV(nfolds=5, rng=123),

    tuning=LatinHypercube(),  #     tuning=Grid(resolution=2), 
    range = r,
    n=200,
    operation=predict_mode, # use for probabalistic 
    measure = fnr,  # use for deterministic or with predict_mode for probab 
    acceleration=CPUThreads(),
)

machbest = machine(
    self_tune,
    x,y)
fit!(machbest,verbosity=1 )

mach = machine(
#     LogisticClassifier(penalty=:l1),
    fitted_params(machbest).best_model,
    x,y)
fit!(mach,verbosity=1 )

┌ Info: Training Machine{ProbabilisticTunedModel{LatinHypercube,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403
┌ Info: Attempting to evaluate 200 models.
└ @ MLJTuning /Users/alexpanchot/.julia/packages/MLJTuning/efiDR/src/tuned_models.jl:685
Evaluating over 200 metamodels:   0%[>                        ]  ETA: N/A┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Eva

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  54%[=============>           ]  ETA: 0:03:05┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinea

Evaluating over 200 metamodels:  69%[=================>       ]  ETA: 0:02:04┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinea

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  82%[===

Evaluating over 200 metamodels:  93%[=======================> ]  ETA: 0:00:29┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  94%[=======================> ]  ETA: 0:00:27┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 100

Machine{LogisticClassifier,…} trained 1 time; caches data
  args: 
    1:	Source @738 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @697 ⏎ `AbstractVector{OrderedFactor{2}}`


In [69]:
FileIO.save("lofreq$(full).jld2", Dict("mach"=>mach,"stnd" => stnd,
        "imputemach" => imputemach, 
    ) ; compress = true
)

In [87]:
mach= FileIO.load("vardict.jld2","mach")
[println(i) for i in fitted_params(mach).coefs[sortperm([i[2] for i in fitted_params(mach).coefs])] ];
params(mach)

:ODDRATIO_Vardict => -5.572995041239996
:refbias_Vardict => -4.018203599082705
:pon_pvalue_Vardict => -3.4798755213719206
:NM_Vardict => -3.27234466201038
:SYMBOL_TP53 => -2.8728309364812237
:ALTREF_CT => -2.2911119540177056
:SYMBOL_PPM1D => -2.276203582052932
:ALTREF_AT => -2.1523947549511546
:dust_score_Vardict => -1.7849825429888468
:total_greater_than_min_alt_count_Vardict => -1.4500898882283528
:SN_Vardict => -1.2504862873342273
:HICNT_Vardict => -1.2504808271520687
:gt_AD_alt_Vardict => -1.2504808271520687
:SYMBOL_DNMT3A => -1.2450782983618665
:SYMBOL_ASXL1 => -1.2262381856927536
:HICOV_Vardict => -1.1356531061327937
:gt_AD_ref_Vardict => -1.1252026513890403
:dust_score_5_Vardict => -1.1238445096642249
:dust_score_10_Vardict => -1.0753437606666796
:gt_AD_alt_Vardict_std => -0.5760704213140069
:ALTREF_CG => -0.5671054699527499
:AltRev_Vardict => -0.4834373007009415
:SYMBOL_CHEK2 => -0.4532968334197934
:SYMBOL_JAK2 => -0.017657921163257397
:SYMBOL_SRSF2 => -0.010053565606153487
:FI

(lambda = 0.010050251256281407,
 gamma = 0.6934673366834171,
 penalty = :l2,
 fit_intercept = true,
 penalize_intercept = false,
 solver = nothing,)

52-element Vector{Int64}:
  4
 31
 19
  9
 43
 47
 40
 50
 20
 25
  5
 10
 12
  ⋮
 26
 21
 15
 16
  8
 17
 39
  7
 45
  2
 49
  1